In [3]:
import pickle

single_run = None
with open('data/single_run.pkl', 'rb') as rf:
    single_run = pickle.load(rf)

with open('data/n_run.pkl', 'rb') as rf:
    n_runs = pickle.load(rf)

In [4]:
chromo_set_1, score_set_1 = single_run

In [5]:
n_results = n_runs

In [6]:
import numpy as np

# Collect Common indexes from res of single run evolution

In [7]:
chromo_len_in_each_gen = []
for i in chromo_set_1:
    chromo_len_in_each_gen.append(np.count_nonzero(i==1))
chromo_len_in_each_gen = np.array(chromo_len_in_each_gen)
chromo_len_in_each_gen

array([100, 100, 338, 480, 480, 480, 480, 480, 480, 375, 375, 666, 666,
       666, 666, 666, 666, 666, 534, 534, 534, 534, 576, 709, 709, 709,
       709, 709, 709, 709])

In [8]:
common_index = dict()
for chromo in chromo_set_1:
    for i in range(len(chromo)):
        if chromo[i] == 1:
            common_index[i] = common_index.setdefault(i, 0) + 1

sorted_common_index = np.array(sorted(common_index.items(), key=lambda x: x[1], reverse=True)[:200])
sorted_common_index[:, 0]

array([ 156,  594,  779,   81,  179,  220,  272,  723, 1190, 1343, 1389,
       1432,   13,  858,   79,  458,  517,  603, 1057, 1070, 1195, 1470,
        744,  869, 1376,  732,  902, 1100,  151,  167,  170,  422,  979,
       1182, 1248,  866, 1064, 1305, 1319,  564,  535,  565,  623,  929,
        149,  251,  261, 1033, 1465,  176,  207,  338,  361,  407,  419,
        432,  495,  515,  525,  532,  590,  713,  725,  750,  754, 1165,
       1180, 1246, 1321, 1375, 1510, 1157, 1467,  104,  113,  187,  288,
        523,  897, 1292, 1496,  938, 1000, 1317,  304,  519,  599,  656,
        693,  896,  915,  997, 1052, 1264,  101,  139,  198,  218,  253,
        279,  283,  313,  352,  790,  967, 1115, 1130, 1132, 1172, 1213,
       1255, 1294, 1330, 1335, 1354, 1380,   23,  102,  105,  490,  544,
        583,  619,  734, 1454,  126,  178,  186,  188,  355,  357,  425,
        868, 1023, 1096, 1120, 1129, 1136, 1137, 1140, 1204, 1262, 1309,
       1348, 1360, 1502,   33,   46,   65,  426,  4

In [16]:
from n_run_evolutions import logmodel, X_train, Y_train, X_test, Y_test, accuracy_score

[nltk_data] Downloading package punkt to /Users/devasenan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/devasenan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/devasenan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/devasenan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [17]:
logmodel.fit(X_train[:,sorted_common_index[:, 0]],Y_train)         
predictions = logmodel.predict(X_test[:,sorted_common_index[:, 0]])
accuracy_score(Y_test,predictions)

0.688

In [ ]:
common_index = []
for chromo in chromo_set_1:
    index = []
    for i in range(len(chromo)):
        if chromo[i] == 1:
            index.append(i)
    common_index.extend(index)

print(len(set(common_index)))

logmodel.fit(X_train[:,common_index],Y_train)         
predictions = logmodel.predict(X_test[:,common_index])
accuracy_score(Y_test,predictions)